## Notebook to prep FOUNDIN-PD modalities for analyses
will output a scaled and covariate adjusted file for full dataset; ie across days
will also output unprepped data just split by differentiation day

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_csv, read_pickle, DataFrame
from dask.dataframe import read_csv as dd_read_csv
import nb_util_funcs as nuf
# from numpy import cumsum
import concurrent.futures
from random import sample
from seaborn import distplot
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as smm
from scipy.stats import f_oneway
from matplotlib.pyplot import rc_context

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

#### set notebooks variables

In [ ]:
# parameters
modality = ''

In [ ]:
# naming
cohort = 'foundin'
set_name = f'{cohort}_daALL_{modality}'

# directories
wrk_dir = f'/home/gibbsr/working/foundin/foundin_qtl'
quants_dir = f'{wrk_dir}/quants'
info_dir = f'{wrk_dir}/sample_info'

# in files
if modality == 'PDUI':
    quants_file = f'{quants_dir}/{set_name}.csv'
    features_file = f'{quants_dir}/{set_name}.features.csv'
elif modality == 'ATAC':
    quants_file = f'{quants_dir}/{set_name}.csv'
    features_file = f'{quants_dir}/{cohort}_consensus_peaks.saf'
elif modality == 'METH':
    quants_file = f'{quants_dir}/{modality}.FINAL_normalized_FOUNDIN_october2020.txt.gz'
    features_file = f'{quants_dir}/EPIC_annotation_hg38.txt'    
covariates_file = f'{info_dir}/{cohort}_{modality}_sample_info.csv'

# out files
all_quants_file = f'{quants_dir}/{set_name}.hdf5'
umap_covs_file = f'{info_dir}/{set_name}.umap.covs.csv'
scaled_file = f'{quants_dir}/{set_name}.scaled.hdf5'
adj_quants_file = f'{quants_dir}/{set_name}.scaled.adj.hdf5'

# constants
if modality == 'METH':
    max_missing_rate = 0.75
else:
    max_missing_rate = 0.25
min_ppscore = 0.05
min_pearson = 0.22
DEBUG = False
low_var_quartile = '75%'
dpi_value = 50

other_id_columns = ['sampleid', 'cdi', 'PPMI_ID', 'DZNE_Barcode', 'DZNE_ID', 
                    'participant_id', 'wgsid', 'PATNO', 'Barcode_LNG', 
                    'Barcode_DZNE', 'Alternate MRN', 'IID', 'FID']
exclude_addl_info_cols = ['data_split', 'ENSG00000188906.15', 'ENSG00000131979.18',
                          'ENSG00000129003.17', 'ENSG00000069329.17', 
                          'ENSG00000177628.15', 'ENSG00000158828.7',
                          'ENSG00000145335.15', 'ENSG00000164535.14', 
                          'ENSG00000165092.12', 'ENSG00000147133.15',
                          'ENSG00000155961.4']
# to match geno's use PPMI3966 Batch3
replace_id_dict = {'PPMI3966B3': 'PPMI3966'}

### load input data

#### load the quantified features matrix and save as hdf5

In [ ]:
%%time
quants_df = read_csv(quants_file, index_col=0)
print(f'shape of input matrix {quants_df.shape}')
# now save the quant matrix in faster file type
nuf.write_df_to_hdf(quants_df, all_quants_file)

if DEBUG:
    display(quants_df.head())

#### split name index to find info

In [ ]:
id_parts = quants_df.index.str.split('_', expand=True).to_frame()
id_parts.columns = ['assay', 'sampleid', 'day']
# id_parts['fullassayid'] = quant_df.index
id_parts['assayid'] = id_parts['assay'] + '_' + id_parts['sampleid'] + '_' + id_parts['day']
print(id_parts.shape)
if DEBUG:
    display(id_parts.sample(5))

#### get counts by day

In [ ]:
days = id_parts['day'].value_counts()
display(days)

#### load covariates files

In [ ]:
covs_df = read_csv(covariates_file, index_col=0)
# drop any duplicated indices
print(covs_df.shape)
covs_df = covs_df[~covs_df.index.duplicated(keep='first')]
print(covs_df.shape)
if DEBUG:
    display(covs_df.sample(5))

In [ ]:
# check for any unexpected samples; ie probably name frmt issue
set(quants_df.index) - set(covs_df.index)

In [ ]:
set(id_parts['sampleid']) - set(covs_df['sampleid'])

#### for further analysis remove the ID columns

In [ ]:
print(covs_df.shape)
cols_to_keep = set(covs_df.columns) - set(other_id_columns) - set(exclude_addl_info_cols)
covs_df = covs_df[cols_to_keep]
print(covs_df.shape)
# display(covs_df.head())

#### load feature annotations

In [ ]:
if modality == 'PDUI':
    features_df = read_csv(features_file)
    features_df.rename(columns={'Loci': 'feature'}, inplace=True)
elif modality == 'METH':
    features_df = read_csv(features_file, sep='\t', header=None)
    features_df.columns = ['Chr', 'start', 'end', 'feature']
elif modality == 'ATAC':
    features_df = read_csv(features_file, sep='\t')
    features_df.columns = ['feature', 'chrom', 'start', 'end', 'strand']    
print(features_df.shape)
if DEBUG:
    display(features_df.head())

### save the quantified features matrix split by day

In [ ]:
%%time
# run the saves in parallel    
with concurrent.futures.ThreadPoolExecutor() as tpe:
    for day in days.index:
        day_df = id_parts.loc[id_parts['day'] == day]
        this_quant_df = quants_df[quants_df.index.isin(day_df['assayid'])]
        print(f'{cohort} {day} {this_quant_df.shape}')
        cohort_quant_filename = f'{quants_dir}/{cohort}_{day}_{modality}.hdf5'
        tpe.submit(nuf.write_df_to_hdf, this_quant_df, cohort_quant_filename)    

#### find IDs for features on sex chromosomes, for dropping later

In [ ]:
sex_chr_feature_ids = features_df.loc[features_df.chrom
                                      .isin(['chrX', 'chrY'])]['feature'].unique()
print(len(sex_chr_feature_ids))

### check expected sex of samples

In [ ]:
#Vawter MP, Evans S, Choudary P et al. Gender-specific gene expression in 
#post-mortem human brain: localization to sex chromosomes. 
#Neuropsychopharmacology 2004;29:373–84.

if modality == 'ATAC':
    sex_specific_features = ['chrX_73852329_73852963', 'chrY_2841364_2842239', 
                             'chrY_19744015_19745452', 'chrY_13478234_13480597', 
                             'chrY_12904296_12906267', 'chrY_12661424_12663659']
elif modality == 'METH':
    sex_specific_features = features_df.loc[features_df['Chr']
                                          .isin(['chrX', 'chrY'])]['feature'].unique()
elif modality == 'PDUI':
    sex_genes = ['XIST','RPS4Y1','RPS4Y2','KDM5D','UTY','DDX3Y','USP9Y']
    sex_features = features_df.loc[features_df.Gene.isin(sex_genes)]
    sex_specific_features = sex_features.feature.to_list()
else:
    sex_specific_features = ['XIST','RPS4Y1','RPS4Y2','KDM5D','UTY','DDX3Y','USP9Y']
sex_features_present = list(set(sex_specific_features) & set(quants_df.columns))
print(f'found {len(sex_features_present)} sex features: \n{sex_features_present}')
quants_sex_df = quants_df[sex_features_present].copy()
print(f'sex features matrix shape {quants_sex_df.shape}')

In [ ]:
%%time
sex_umap_df = nuf.generate_umap_covs_df(quants_sex_df, covs_df)
nuf.plot_umap_clusters(sex_umap_df, hue_cov='sex', style_cov='day')

### calculate, plot detection rates and subset well detected features

In [ ]:
%%time
trait_miss_rates, sample_miss_rates = nuf.calculate_detection_rates(quants_df, modality)
nuf.plot_missing_rates(trait_miss_rates, sample_miss_rates)
bad_call_rate_features = nuf.bad_callrate_features(trait_miss_rates, max_missing_rate)
quants_wd_df = nuf.subset_well_detected_features(quants_df, bad_call_rate_features)

### standardize the dataset using transform

In [ ]:
%%time
traits_scaled_df = nuf.scale_dataframe(quants_wd_df)

In [ ]:
# check transformation for random feature
nuf.plot_trnsfrm_effect_example(quants_df, traits_scaled_df,
                                bf_label=modality, 
                                af_label='quantile transformed')

### save scaled, well detected data for all days

In [ ]:
nuf.write_df_to_hdf(traits_scaled_df, scaled_file)

### generate covariates from variance

#### take a look at variance in data, assuming mostly driven by d0 -> d25, ie IPSc -> differentiating neurons

#### exclude low variance features from covariate generation

In [ ]:
quants_var_df = nuf.exclude_low_var_features(traits_scaled_df, 
                                             quartile_to_drop=low_var_quartile)

In [ ]:
variance_features = set(quants_var_df.columns) - (set(sex_specific_features))
print(len(variance_features))

### take a look at the data

#### not going to use PCs but take a look at PCA anyway

In [ ]:
pcs_df = nuf.generate_pca(quants_var_df[variance_features], plot_variance=True)
print(pcs_df.shape)
if DEBUG:
    display(pcs_df)

In [ ]:
%%time
pcs_df = pcs_df.merge(covs_df, how='left', left_index=True, right_index=True)
# since just checking the PCs and not using just run ppscore on 1st three
covs_target_list = ['PC_0', 'PC_1', 'PC_2']
covs_to_check = nuf.pps_predict_targets(pcs_df, covs_target_list)
nuf.plot_ppscore_matrix(pcs_df, covs_to_check, covs_target_list)

In [ ]:
nuf.plot_pca_pair(pcs_df, 'PC_0', 'PC_1', hue_cov='day', style_cov='Batch')

#### generate unknown covariates and see if know covariates are source of variations

In [ ]:
%%time
umap_df = nuf.generate_umap_covs_df(quants_var_df[variance_features], covs_df)
covs_target_list = ['x_umap', 'y_umap']
covs_to_check = nuf.pps_predict_targets(umap_df, covs_target_list)
nuf.plot_ppscore_matrix(umap_df, covs_to_check, covs_target_list)
if len(covs_to_check) > 0:
    umap_dums_covs_df = nuf.dummy_covs_as_needed(umap_df[(set(covs_to_check) | 
                                                          set(covs_target_list))])
    nuf.plot_correlation_heatmap(umap_dums_covs_df)
    nuf.plot_correlation_heatmap(umap_dums_covs_df, covs_target_list)

#### plot umap of with known covariates of interest

In [ ]:
nuf.plot_umap_clusters(umap_df, hue_cov='day', style_cov='Batch')

#### do quick anova by day to identify features changed with cell differentiation 

this is since we know differention should be largest source of variation, so figure out which features to exclude to get around

In [ ]:
# split samples by group (day)
feats_by_day = {}
for day in days.index:
    day_df = id_parts.loc[id_parts['day'] == day]
    this_quant_df = traits_scaled_df[traits_scaled_df.index.isin(day_df['assayid'])]
    feats_by_day[day] = this_quant_df
    print(f'{cohort} {day} {this_quant_df.shape}')

# calculate one-way ANOVA for the groups
if modality == 'METH':
    fvalues, pvalues = f_oneway(feats_by_day.get('da0'),  
                                      feats_by_day.get('da65'))    
else:
    fvalues, pvalues = f_oneway(feats_by_day.get('da0'), 
                                      feats_by_day.get('da25'), 
                                      feats_by_day.get('da65'))

# make df from results
anova_results_df = DataFrame(data={'fvalues': fvalues, 'pvalues': pvalues}, 
                                index=traits_scaled_df.columns)
# apply a B&H FDR to pvalues
anova_results_df['bh_fdr'] = smm.fdrcorrection(anova_results_df.pvalues.fillna(1))[1]

print(anova_results_df.shape)
if DEBUG:
    display(anova_results_df.head())

In [ ]:
anova_results_df.loc[anova_results_df['bh_fdr'] < 0.05].shape

#### determine final set of features to use for variance detection
exluding bottom 25% variance features, sex features, tissue elevated features, and cell differentiation features

In [ ]:
no_change_features = anova_results_df.loc[anova_results_df['bh_fdr'] > 0.05].index.values
print(len(no_change_features))

no_change_variance_features = (set(no_change_features) & set(quants_var_df.columns)) - set(sex_chr_feature_ids)
print(len(no_change_variance_features))

#### generate unknown covariates from final set of features for variance detection

#### check PCAs again, still not using

In [ ]:
pcs_df = nuf.generate_pca(quants_var_df[no_change_variance_features], plot_variance=True)
print(pcs_df.shape)
if DEBUG:
    display(pcs_df)

In [ ]:
%%time
pcs_df = pcs_df.merge(covs_df, how='left', left_index=True, right_index=True)
# since just checking the PCs and not using just run ppscore on 1st three
covs_target_list = ['PC_0', 'PC_1', 'PC_2']
covs_to_check = nuf.pps_predict_targets(pcs_df, covs_target_list)
nuf.plot_ppscore_matrix(pcs_df, covs_to_check, covs_target_list)

In [ ]:
nuf.plot_pca_pair(pcs_df, 'PC_0', 'PC_1', hue_cov='day', style_cov='Batch')

In [ ]:
%%time
umap_df = nuf.generate_umap_covs_df(quants_var_df[no_change_variance_features], covs_df)
covs_target_list = ['x_umap', 'y_umap']
covs_to_check = nuf.pps_predict_targets(umap_df, covs_target_list)
nuf.plot_ppscore_matrix(umap_df, covs_to_check, covs_target_list)
if len(covs_to_check) > 0:
    umap_dums_covs_df = nuf.dummy_covs_as_needed(umap_df[(set(covs_to_check) | 
                                                          set(covs_target_list))])
    nuf.plot_correlation_heatmap(umap_dums_covs_df)
    nuf.plot_correlation_heatmap(umap_dums_covs_df, covs_target_list)

#### plot umap of with known covariates of interest

In [ ]:
nuf.plot_umap_clusters(umap_df, hue_cov='day', style_cov='Batch')

In [ ]:
nuf.plot_umap_clusters(umap_df, hue_cov='Batch', size_cov='DopaminergicNeurons')

### keep created covars and save them

In [ ]:
# standardize the covariates
umap_covs_df = nuf.scale_dataframe(umap_df[covs_target_list])
# now save the covariates
umap_covs_df.to_csv(umap_covs_file)

### adjust the scaled data by the covariates

In [ ]:
# check to see in df's have same indices
if not traits_scaled_df.index.equals(umap_covs_df.index):
    print('indices are not equal re-index')
    umap_covs_df.reindex(traits_scaled_df.index)
    
traits_scaled_df.index.equals(umap_covs_df.index)    

In [ ]:
%%time

residuals_df, cov_scores_df = nuf.covariate_residuals(traits_scaled_df, umap_covs_df)

#take a peek at the data
print(residuals_df.shape)
print(cov_scores_df.shape)

# print(cov_scores_df.head())

In [ ]:
# get a summary of the covariates model scores
print(cov_scores_df.describe())
# look at the distribution of covariate model scores, 
# ie get a sense any feature driven by covariates
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': dpi_value}):
    plt.style.use('seaborn-bright')
    distplot(cov_scores_df['score'])
    plt.show()

#### remove any features that had more than 75% score

In [ ]:
drop_features = cov_scores_df[cov_scores_df.score > 0.75].index.values
keep_features = list(set(residuals_df.columns) - set(drop_features))
residuals_df = residuals_df[keep_features]
cov_scores_df = cov_scores_df.loc[cov_scores_df.index.isin(keep_features)]
print(len(drop_features))
print(len(keep_features))
print(residuals_df.shape)
print(cov_scores_df.shape)

### save scaled and covariate adjusted data

In [ ]:
%%time 

residuals_df.to_hdf(adj_quants_file, key='quants', mode='w')

#### take a look at the scaled and covariate adjusted data

In [ ]:
nuf.plot_trnsfrm_effect_example(traits_scaled_df, residuals_df)

In [ ]:
# find feature with largest score
large_adj_trait = cov_scores_df.loc[cov_scores_df['score'] == max(cov_scores_df['score'])]
print(large_adj_trait)
large_adj_traid_id = large_adj_trait.index.values[0]

# spot check same feature with largest adjustment effect
nuf.plot_trnsfrm_effect_example(traits_scaled_df, residuals_df, large_adj_traid_id)

#### what are the post scaled and covariate adjusted umap variables correlated with

In [ ]:
%%time
umap_df = nuf.generate_umap_covs_df(residuals_df, covs_df)
covs_target_list = ['x_umap', 'y_umap']
covs_to_check = nuf.pps_predict_targets(umap_df, covs_target_list)
nuf.plot_ppscore_matrix(umap_df, covs_to_check, covs_target_list)
if len(covs_to_check) > 0:
    umap_dums_covs_df = nuf.dummy_covs_as_needed(umap_df[(set(covs_to_check) | 
                                                          set(covs_target_list))])
    nuf.plot_correlation_heatmap(umap_dums_covs_df)
    nuf.plot_correlation_heatmap(umap_dums_covs_df, covs_target_list)

In [ ]:
nuf.plot_umap_clusters(umap_df, hue_cov='day', style_cov='Batch')

In [ ]:
!date